# Colour prediction for a MTG card image

---

### Import necessary libraries

In [1]:
!sudo pip install h5py
import h5py
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy import misc
import glob
from google.colab import drive
import pickle
import pandas as pd
from sklearn import preprocessing
import collections

### Mount Google drive

In [2]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Unzip the data

In [3]:
!unzip /content/gdrive/'My Drive'/'Magic Final Project'/data/images/cropped.zip

Archive:  /content/gdrive/My Drive/Magic Final Project/data/images/cropped.zip
replace cropped/39934090-36a6-4183-9176-97ea932d2685.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
df = pd.read_csv("gdrive/My Drive/Magic Final Project/data/cards.csv")

In [0]:
df = df.sort_values('Unnamed: 0')
df = df.reset_index(drop=True)

In [0]:
df1 = pd.DataFrame(df['colors'][:23000])
mask = (df1['colors'].str.len() == 1)
df1 = df1.loc[mask]

In [0]:
sgl_clr_exists= np.array(df1.index)

In [8]:
collections.Counter(df1['colors'])

Counter({'B': 3255, 'G': 3235, 'R': 3270, 'U': 3287, 'W': 3253})

In [0]:
le = preprocessing.LabelEncoder()
le.fit(df1['colors'])
df1['color_code'] = le.transform(df1['colors'])

In [0]:
image_labels_val = list(df1['color_code'])

In [0]:
image_names = list(df.iloc[sgl_clr_exists,:]['Unnamed: 0'])
image_labels1 = list(df1['color_code'])

In [12]:
print(len(image_names), len(image_labels1))

16300 16300


In [0]:
df1 = pd.DataFrame(df['colors'][23000:])
mask = (df1['colors'].str.len() == 1)
df1 = df1.loc[mask]

In [0]:
sgl_clr_exists= np.array(df1.index)

In [32]:
collections.Counter(df1['colors'])

Counter({'B': 214, 'G': 221, 'R': 211, 'U': 215, 'W': 233})

In [0]:
le = preprocessing.LabelEncoder()
le.fit(df1['colors'])
df1['color_code'] = le.transform(df1['colors'])

In [0]:
image_names_val = list(df.iloc[sgl_clr_exists,:]['Unnamed: 0'])
image_labels_test = list(df1['color_code'])

In [18]:
print(len(image_names_val), len(image_labels_test))

1094 1094


In [0]:
data = np.zeros((16300, 452, 616, 3), dtype=np.dtype('u1'))
i = 0
for name in image_names:
    img = "cropped/"+name+".jpg"
    data[i] = np.array(Image.open(img))
    i += 1


In [0]:
test_data = np.zeros((1094, 452, 616, 3), dtype=np.dtype('u1'))
i = 0
for name in image_names_val:
    img = "cropped/"+name+".jpg"
    test_data[i] = np.array(Image.open(img))
    i += 1


In [0]:
def conc(l):
    return tf.keras.backend.concatenate((l[0], l[1]), axis=-1)

In [26]:
inp_k = tf.keras.layers.Input((452, 616, 3))
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(452, 616, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, (3,3), strides=(2,2), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, (3,3), strides=(2,2), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu')
])

m = model(inp_k)

test_model=tf.keras.models.Sequential([
    tf.keras.layers.GlobalAveragePooling2D(input_shape=((452, 616, 3)))
])
t = test_model(inp_k)

lam = tf.keras.layers.Lambda(conc, output_shape=(515,))([t, m])
intm = tf.keras.layers.Dense(64, activation='relu')(lam)
out = tf.keras.layers.Dense(5, activation='softmax')(intm)
Model = tf.keras.Model(inp_k, out)

opti = tf.train.AdamOptimizer(learning_rate=0.001)
Model.compile(optimizer=opti, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
Model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 452, 616, 3) 0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 3)            0           input_1[0][0]                    
__________________________________________________________________________________________________
sequential (Sequential)         (None, 512)          4378048     input_1[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 515)          0           sequential_1[1][0]               
             

In [27]:
mc = tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
history = Model.fit(data, image_labels1, validation_split=0.05, batch_size=100, epochs=30, callbacks=[mc])

Train on 15485 samples, validate on 815 samples
Epoch 1/30
15400/15485 [============================>.] - ETA: 0s - loss: 4.5394 - acc: 0.3382
Epoch 00001: val_acc improved from -inf to 0.39877, saving model to best_model.h5
15485/15485 [==============================] - 65s 4ms/sample - loss: 4.5225 - acc: 0.3388 - val_loss: 1.4541 - val_acc: 0.3988
Epoch 2/30
15400/15485 [============================>.] - ETA: 0s - loss: 1.4217 - acc: 0.4062
Epoch 00002: val_acc did not improve from 0.39877
15485/15485 [==============================] - 58s 4ms/sample - loss: 1.4219 - acc: 0.4064 - val_loss: 1.5409 - val_acc: 0.3546
Epoch 3/30
15400/15485 [============================>.] - ETA: 0s - loss: 1.3573 - acc: 0.4497
Epoch 00003: val_acc improved from 0.39877 to 0.40245, saving model to best_model.h5
15485/15485 [==============================] - 58s 4ms/sample - loss: 1.3576 - acc: 0.4493 - val_loss: 1.4627 - val_acc: 0.4025
Epoch 4/30
15400/15485 [============================>.] - ETA: 0s 

In [28]:
Model = tf.keras.models.load_model('best_model.h5')
Model.compile(optimizer=opti, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [35]:
Model.evaluate(test_data, image_labels_test)

1094/1094 [==============================] - 2s 2ms/sample - loss: 1.4792 - acc: 0.3784


[1.4792107034209006, 0.37842777]